# DCR Graphs in PM4Py

This tutorial overviews a simple workflow one might have when working with DCR graphs.

We first import an event log.

In [9]:
import pm4py
log = pm4py.read_xes('data/running_example.xes')

parsing log, completed traces :: 100%|██████████| 6/6 [00:00<00:00, 6016.21it/s]


The next step is obtaining a process model from this event log using the discovery algorithm.

In [11]:
dcr, _ = pm4py.discover_dcr(log)


From here, one can perform some conformance checking and be presented with a nice diagnostics dataframe.

In [15]:
conf_res = pm4py.conformance_dcr(log, dcr, return_diagnostics_dataframe=True)
print(conf_res)

  case_id  no_dev_total  no_constr_total  dev_fitness
0       3             5               31     0.838710
1       2             4               31     0.870968
2       1             4               31     0.870968
3       6             4               31     0.870968
4       5             4               31     0.870968
5       4             4               31     0.870968


c:\Users\herma\pm4py-dcr\pm4py\utils.py:510: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


Finally, we can also visualise the mined DCR graph by exporting it as an 'xml' file and then viewing it at the corresponding portal.

In [16]:
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
path = "C:/Users/herma/Downloads/dcr-graph.xml"
pm4py.write_dcr_xml(dcr, path, dcr_exporter.DCR_JS_PORTAL, 'dcr-graph')

In this case, we can visualise the graph at the following link: https://hugoalopez-dtu.github.io/dcr-js/